### Disclaimer

Distribution authorized to U.S. Government agencies and their contractors. Other requests for this document shall be referred to the MIT Lincoln Laboratory Technology Office.

This material is based upon work supported by the Under Secretary of Defense for Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions, findings, conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the Under Secretary of Defense for Research and Engineering.

© 2019 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part 252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed above. Use of this work other than as specifically authorized by the U.S. Government may violate any copyrights that exist in this work.


# Quadrotor Demostration
This notebook introduces the Python interface for controlling a quadrotor agent in TESSE simulation. This is implemented for VNAV 2020 Fall class.

In [ ]:
# load packages
from tesse.env import Env
from tesse.msgs import *

import time
import matplotlib.pyplot as plt
import defusedxml.ElementTree as ET
import numpy as np

In [ ]:
# initialize connections
env = Env()

In [ ]:
# reset quadrotor to origin
# note: Unity uses left-handed frame with y axis pointing up
def reset_to_origin():
    env.send(Reposition(0, 1, 0, 0, 0, 0, 0))
    
reset_to_origin()

In [ ]:
# Quadrotor test 1: control individual props by setting each one of them to a positive value 
# for a short period of time before resetting
def turn_prop_then_stop(idx):
    speeds = 4*[0]
    speeds[idx] = 80
    env.send(PropSpeeds(speeds[0], speeds[1], speeds[2], speeds[3]))
    time.sleep(1)
    env.send(PropSpeeds(0,0,0,0))
    time.sleep(1)
    s
turn_prop_then_stop(0)
reset_to_origin()
time.sleep(3)

turn_prop_then_stop(1)
reset_to_origin()
time.sleep(3)

turn_prop_then_stop(2)
reset_to_origin()
time.sleep(3)

turn_prop_then_stop(3)
reset_to_origin()
time.sleep(3)


In [ ]:
# Quadrotor test 2: full throttle up
def full_throttle_up(speed):
    env.send(PropSpeeds(speed, speed, speed, speed))
    time.sleep(3)
    env.send(PropSpeeds(0,0,0,0))
    
full_throttle_up(150)
time.sleep(3)

In [ ]:
# Quadrotor test 3: IMU (metadata while flying)
# 10 seconds timeout
timeout = time.time() + 2
speed = 70
time_data = []
x_data = []
y_data = []
z_data = []

# Fly up
while True:
    if time.time() > timeout:
        break
    env.send(PropSpeeds(speed, speed, speed, speed))
    time.sleep(0.005)
    response = env.request(MetadataRequest())
    if response is not None:
        tree = ET.fromstring(response.metadata)
        x_data.append(float(tree.find('position').attrib['x']))
        y_data.append(float(tree.find('position').attrib['y']))
        z_data.append(float(tree.find('position').attrib['z']))
        time_data.append(float(tree.find('time').text))
        
# Stop all props
env.send(PropSpeeds(0, 0, 0, 0))

timeout = time.time() + 6
# Record data while falling down
while True:
    if time.time() > timeout:
        break
    time.sleep(0.005)
    response = env.request(MetadataRequest())
    if response is not None:
        tree = ET.fromstring(response.metadata)
        x_data.append(float(tree.find('position').attrib['x']))
        y_data.append(float(tree.find('position').attrib['y']))
        z_data.append(float(tree.find('position').attrib['z']))
        time_data.append(float(tree.find('time').text))
        
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)

ax1.plot(time_data, x_data)
ax1.set_ylabel("x (m)")
ax1.set_title('Position Data with Time')

ax2.plot(time_data, y_data)
ax2.set_ylabel("y (m)")

ax3.plot(time_data, z_data)
ax3.set_xlabel("Time (s)")
ax3.set_ylabel("z (m)")
